In [1]:
import datetime as dt

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

In [2]:
pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

In [3]:
import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

In [16]:
wave = pd.read_csv('MACD_Combined.csv')
nlc = pd.read_csv('NLC.csv')
hammer = pd.read_csv('Hammer.csv')

In [17]:
wave = wave[wave['Trade Year'] >= 2019]

wave['STRATEGY'] = 'WAVE'
nlc['STRATEGY'] = 'NLC'
hammer['STRATEGY'] = 'HAMMER'

wave['PORTFOLIO_VALUE'] = 100000
nlc['PORTFOLIO_VALUE'] = 1000000
hammer['PORTFOLIO_VALUE'] = 1000000

In [18]:
wave['DATETIME'] = wave['Entry Time']

In [19]:
wave['ROI%'] = wave['ROI% PF']

In [20]:
combined_df = pd.concat([wave, nlc, hammer], ignore_index=True)

In [21]:
combined_df = combined_df.sort_values(by='DATETIME')

In [22]:
combined_df.drop(columns=['DD', 'Margin', 'Cumulative ROI%', 'DD%'], inplace=True)

In [23]:
combined_df['Final ROI%'] = combined_df['ROI%'] / 3

In [24]:
combined_df["DD%"] = combined_df["Final ROI%"].cumsum() - combined_df["Final ROI%"].cumsum().cummax()

In [29]:
combined_df.tail(350)

,Signal Generated At,Trade Type,Entry Time,Entry Price,Initial SL,Final SL,Exit Time,Exit Price,Points Captured,After Costs,PnL,Remarks,Qty,Leverage,ROI%,Trade Year,Trade Month,Index,Portfolio,Allocation,Qty apAlloc,PnL apAlloc,ROI% PF,STRATEGY,PORTFOLIO_VALUE,DATETIME,date,day,expiry,DTE,strike,type,initial sl,Exit date,RSI on Entry,Remark,Slippage,Highest High,Lowest Low,Max ROI%,Max Cumulative ROI%,Final ROI%,DD%
2164,NaN,NaN,09:25:00,21.7500,NaN,NaN,15:25:00,0.0500,21.7000,NaN,168486.2745,NaN,7843.1373,NaN,1.6849,2024,11,NaN,NaN,NaN,NaN,NaN,NaN,NLC,1000000,2024-11-07 09:25:00,2024-11-07,Thursday,2024-11-07,0.0000,24500.0000,C,60.0000,2024-11-07,0.0000,EOD exit,0.2180,30.8000,0.0500,1.7020,529.5548,0.5616,-1.2173
1413,2024-11-07 10:15:00,LONG,2024-11-07 10:45:00,51887.6500,51109.3352,51109.3352,2024-11-07 10:45:00,51957.2500,69.6000,59.2155,5329.3959,MACD Reversal,90.0000,5.0000,0.1332,2024,11,BANKNIFTY,1000000.0000,250000.0000,22.5000,1332.3490,0.1332,WAVE,100000,2024-11-07 10:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0444,-1.1729
1414,2024-11-07 11:15:00,LONG,2024-11-07 12:15:00,24233.2000,24073.2609,24073.2609,2024-11-07 13:15:00,24216.0000,-17.2000,-26.8898,-8066.9520,MACD Reversal,300.0000,7.0000,-0.2017,2024,11,NIFTY,1000000.0000,250000.0000,75.0000,-2016.7380,-0.2017,WAVE,100000,2024-11-07 12:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.0672,-1.2401
1415,2024-11-07 11:15:00,LONG,2024-11-07 12:15:00,79605.1800,79127.5489,79127.5489,2024-11-08 09:15:00,79127.5489,-477.6311,-493.5044,-49350.4353,Initial SL Hit,100.0000,8.0000,-1.2338,2024,11,SENSEX,1000000.0000,250000.0000,25.0000,-12337.6088,-1.2338,WAVE,100000,2024-11-07 12:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.4113,-1.6514
3550,NaN,NaN,09:15:00,116.6500,NaN,NaN,09:45:00,172.9500,-56.3000,NaN,-21028.7744,NaN,355.2398,NaN,-2.1029,2024,11,NaN,NaN,NaN,NaN,NaN,NaN,HAMMER,1000000,2024-11-08 09:15:00,2024-11-08,Friday,2024-11-14,6.0000,24250.0000,C,172.9500,2024-11-08,17.0167,SL hit,2.8960,215.0000,115.8500,0.0284,NaN,-0.7010,-2.3524
3551,NaN,NaN,09:45:00,177.5000,NaN,NaN,15:25:00,0.0500,177.4500,NaN,52051.7037,NaN,296.2963,NaN,5.2052,2024,11,NaN,NaN,NaN,NaN,NaN,NaN,HAMMER,1000000,2024-11-08 09:45:00,2024-11-08,Friday,2024-11-14,6.0000,24200.0000,C,245.0000,2024-11-14,47.9076,EOD exit,1.7755,221.0000,0.0500,5.2578,NaN,1.7351,-0.6173
3552,NaN,NaN,10:05:00,151.0500,NaN,NaN,09:15:00,247.2500,-96.2000,NaN,-20828.0665,NaN,207.9002,NaN,-2.0828,2024,11,NaN,NaN,NaN,NaN,NaN,NaN,HAMMER,1000000,2024-11-08 10:05:00,2024-11-08,Friday,2024-11-14,6.0000,24200.0000,P,247.2500,2024-11-11,48.9182,SL hit,3.9830,258.5000,143.3000,0.1611,NaN,-0.6943,-1.3116
3553,NaN,NaN,10:15:00,84.3000,NaN,NaN,13:15:00,168.1000,-83.8000,NaN,-20602.3866,NaN,238.6635,NaN,-2.0602,2024,11,NaN,NaN,NaN,NaN,NaN,NaN,HAMMER,1000000,2024-11-11 10:15:00,2024-11-11,Monday,2024-11-14,3.0000,24050.0000,P,168.1000,2024-11-12,43.7005,SL hit,2.5240,178.9500,30.9000,1.2745,NaN,-0.6867,-1.9983
1417,2024-11-11 10:45:00,LONG,2024-11-11 11:15:00,52104.5000,51322.9325,51827.3917,2024-11-11 15:15:00,51812.3500,-292.1500,-302.5417,-27228.7517,TSL Hit,90.0000,5.0000,-0.6807,2024,11,BANKNIFTY,1000000.0000,250000.0000,22.5000,-6807.1879,-0.6807,WAVE,100000,2024-11-11 11:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.2269,-2.2252
1416,2024-11-11 10:15:00,LONG,2024-11-11 11:15:00,80029.2300,79549.0546,79559.5058,2024-11-11 13:15:00,79549.0546,-480.1754,-496.1332,-49613.3208,Initial SL hit,100.0000,8.0000,-1.2403,2024,11,SENSEX,1000000.0000,250000.0000,25.0000,-12403.3302,-1.2403,WAVE,100000,2024-11-11 11:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.4134,-2.6387


In [26]:
def generate_stats(tb_expiry):
    stats_df8 = pd.DataFrame(
        index=range(2019, 2026),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe
    
    # Iterate over each year
    for year in range(2019, 2026):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]
    
        # Calculate total ROI
        total_roi = year_trades["Final ROI%"].sum()
    
        # Calculate total number of trades
        total_trades = len(year_trades)
    
        # Calculate win rate
        win_rate = (year_trades["Final ROI%"] > 0).mean() * 100
    
        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["Final ROI%"] > 0]["Final ROI%"].mean()
    
        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["Final ROI%"] < 0]["Final ROI%"].mean()
    
        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["Final ROI%"].cumsum() - year_trades["Final ROI%"].cumsum().cummax()
        ).min()
    
        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        # variation = f'{signal_ma} , {trailing_ma}, {time_of_day}'
    
        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            # variation,
        ]
    
    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["Final ROI%"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["Final ROI%"] > 0]["Final ROI%"].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["Final ROI%"] < 0]["Final ROI%"].mean()
    overall_max_drawdown = (
        combined_df_sorted["Final ROI%"].cumsum() - combined_df_sorted["Final ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    # overall_variation = variation
    
    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        # overall_variation,
    ]
    return pd.DataFrame(stats_df8)

In [27]:
stats = generate_stats(combined_df)
stats

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2019,110.5335,512,41.4062,1.2520,-0.5163,-11.7731,9.3887
2020,170.2309,565,41.5929,1.6645,-0.6695,-10.0630,16.9165
2021,115.5075,545,42.5688,1.3045,-0.5979,-10.7701,10.7248
2022,96.7082,611,39.1162,1.3583,-0.6127,-13.4158,7.2085
2023,80.0977,592,40.0338,1.0626,-0.4838,-9.8880,8.1005
2024,87.0248,597,41.2060,1.1142,-0.5330,-9.7356,8.9388
2025,38.7765,254,40.5512,1.1844,-0.5511,-14.8504,2.6111
Overall,698.8791,3676.0000,40.9140,1.2844,-0.5676,-17.0477,40.9955


## Combined PF : WAVE : NLC : Hammer (1 : 1 : 1)